In [1]:
from mystem import mystem as mstm

In [2]:
import unicodedata
import re

def remove_accents(input_str):
    nfс_form = unicodedata.normalize('NFC', input_str)
    nfс_form = re.sub(r'[^А-Яа-яЁё\s\?\!-]', u'', nfс_form, flags=re.UNICODE)
    return u"".join([c for c in nfс_form if not unicodedata.combining(c)])

In [3]:
# Выделение лемм из выхлопа mystem
def lemmatized_text_list(contextList):
    """
    Gets lemmas from list of tuples (mystem func)
    """
    line = ''
    textList = list()
    for sentence in contextList:
        if len(sentence) == 0:
            continue
        for word in sentence:
            if word[2] in ['UNKNOWN', 'CONJ', 'INTJ', 'PART', 'PR']:
                continue
            lemma = word[1]
            if lemma != '.':
                if len(lemma) > 1 and lemma[-1:] == '?':
                    lemma = lemma[:-1]                     
                line += lemma + ' '
        textList.append(line)
        line = ''
    return textList

In [4]:
def write_to_file(path, filename, sentencesList):
    outputFile = path + filename
    file_obj = open(outputFile, 'w', encoding="utf8")
    file_obj.writelines(sentencesList)
    file_obj.close()

## Обработка НКРЯ (лучшее)

In [5]:
trainList = ['балка',
             'вид',
             'винт',
             'горн',
             'губа',
             'жаба',
             'клетка',
             'крыло',
             'купюра', 
             'курица', 
             'лавка', 
             'лайка', 
             'лев', 
             'лира', 
             'мина', 
             'мишень', 
             'обед', 
             'оклад', 
             'опушка', 
             'полис', 
             'пост', 
             'поток', 
             'проказа', 
             'пропасть', 
             'проспект', 
             'пытка',
             'рысь',
             'среда',
             'хвост',
             'штамп',
            ]

Скачивание xml-файла под каждое слово.

Хотел сделать считывание через поток, но там были с этим проблемы, поэтому так.

In [17]:
import urllib.request
from urllib.parse import quote

for word in trainList:
    testfile = urllib.request.URLopener()
    file_name = 'http://search1.ruscorpora.ru/download-xml.xml?env=alpha&text=lexgramm&mode=main&sort=gr_tagging&lang=ru&nodia=1&parent1=0&level1=0&lex1={}&gramm1=S%2C(nom%7Cgen%7Cgen2%7Cdat%7Cacc%7Cacc2%7Cins%7Cloc%7Cloc2)%2C(sg%7Cpl)%2Cinan&sem-mod1=sem&sem-mod1=sem2&parent2=0&level2=0&min2=1&max2=1&sem-mod2=sem&sem-mod2=sem2&m2=&p=0&dpp=1000&spd=10&spp=1000&out=kwic&dl=xml'.format(quote(word))
    try:
        testfile.retrieve(file_name, "Data//НКРЯ//xml//{}.xml".format(word))
    except:
        print('Error', word)

Выделение текста из каждого xml.

Есть одна проблема. Если брать текст из поля, где содержится текст целиком, то он обрежется, потому что нужное слово выделяется отдельным тегом прямо посреди предложения.

In [6]:
import xml.etree.ElementTree

for word in trainList:
    filepath = 'Data//НКРЯ//xml//' + word + '.xml'
    e = xml.etree.ElementTree.parse(filepath).getroot()

    contextList = list()
    tagPrefix = '{urn:schemas-microsoft-com:office:spreadsheet}'

    Worksheet = e.find(tagPrefix + 'Worksheet')
    Table = Worksheet.find(tagPrefix + 'Table')

    count = 0
    for child in Table:
        if child.tag == tagPrefix + 'Row':
            text = ''
            cells = child.findall(tagPrefix + 'Cell')
            cell = cells[-1]
            data = cell.find(tagPrefix + 'Data')
            try:
                text += data.text
            except:
                pass
            text += word + ' '
            cell = cells[5]
            data = cell.find(tagPrefix + 'Data')
            try:
                text += data.text
            except:
                pass

            if text != 'Full context' + word + ' Right context':
                contextList.append(text + '\n')

    write_to_file('Data//НКРЯ//txt//', word + '.txt', contextList)

Отчистка текстов от лишних символов и обработка mystem.

In [13]:
mystemDict = dict()

for word in trainList:
    filename = 'Data//НКРЯ//txt//' + word + '.txt'
    file_obj = open(filename, 'r', encoding="utf8")
    text = remove_accents(file_obj.read())
    mystemDict[word] = mstm(text)
    file_obj.close()

Обработка выхлопа mystem (можно было вставить в прошлый цикл).

In [34]:
contextDict = dict()

for word in mystemDict:
    contextList = mystemDict[word]
    
    # Выделение лемм
    mystemContextList = lemmatized_text_list(contextList)
    
    # Проверка на наличие слова в тексте (mystem мог поделить контекст на несколько предложений)
    mystemContextList = [x + '\n' for x in mystemContextList if x.find(word) != -1]
    
    # Удаление предложений, где меньше 3 слов
    # Стоит цифра 5, потому что из-за особенности форматирования по краям стоят пустые строки
    mystemContextList = [' ' + x for x in mystemContextList if len(x.split(' ')) > 5]
    contextDict[word] = mystemContextList

Подсчет количества предложений и запись.

In [35]:
for word in trainList:
    wordList = contextDict[word]
    print(len(wordList), word)
    write_to_file('Data//НКРЯ//lemma txt//', word + '.txt', wordList)

2077 балка
2859 вид
1927 винт
1637 горн
3331 губа
1137 жаба
3006 клетка
2219 крыло
1544 купюра
1851 курица
2154 лавка
414 лайка
1942 лев
1871 лира
1807 мина
1779 мишень
2322 обед
1631 оклад
2073 опушка
701 полис
2086 пост
1978 поток
892 проказа
1250 пропасть
1698 проспект
1861 пытка
1977 рысь
1899 среда
2204 хвост
1563 штамп


Я считаю, что в данной задаче кроме НКРЯ использовать ничего не нужно, потому что можно сразу получить большой набор предложений с необходимым словом. Код дальше не был использован в итоговой версии.

## Обработка OpenCorpora (часть кода утеряна)

In [2]:
filesList = list()
for i in range(2, 4062):
    number = str(i)
    while len(number) < 4:
        number = '0' + number
    trainFile = './/Data//OpenCorpora//opencorpora original//' + number +'.txt'
    try:
        file_obj = open(trainFile, 'r', encoding="utf8")
        filesList.append(file_obj.read())
        file_obj.close()
    except:
        continue

In [5]:
# здесь была обработка через mystem и сохранение в отдельный файл

In [3]:
textsList = list()
for text in filesList:
    fileSentencesList = text.split('\n')
    sentencesList = list()
    for sentence in fileSentencesList:
        tuplesList = sentence.split(' ||| ')
        tuplesList = tuplesList[:-1]
        sentencesList.append(tuplesList)
    textsList.append(sentencesList)

In [4]:
outputFile = 'Data//OpenCorpora//opencorpora lemmas cleaned//text.txt'
file_obj = open(outputFile, 'w', encoding="utf8")
verbsString = ''
for text in textsList:
    for sentence in text:
        line = ' '
        for wordTuple in sentence:
            wordInfo = wordTuple[:-1]
            wordInfo = wordInfo[1:]
            wordInfo = wordInfo.split(' || ')
            word = wordInfo[0].split(' ')[0]
            mark = wordInfo[1].split(' ')[0]
            if mark in ['ADVB', 'ADJF', 'NOUN', 'VERB', 'INFN']:
                if mark == 'VERB':
                    verbsString = verbsString + word + ' \n'
                line = line + word + ' '
        line = line + '\n'
        file_obj.write(line)
file_obj.close()

В текстах OpenCorpora у глаголов вместо лемм дают форму ед.ч. им. падежа, поэтому их нужно прогонять в Mystem.

In [9]:
mystemVerbs = mstm(verbsString)

In [10]:
print(mystemVerbs[:10])

[[('учу', 'учить', 'V')], [('сохранился', 'сохраняться', 'V')], [('вернулся', 'вернуться', 'V')], [('начинаюсь', 'начинаться', 'V')], [('отличаюсь', 'отличаться', 'V')], [('называюсь', 'называться', 'V')], [('соответствую', 'соответствовать', 'V')], [('выступаю', 'выступать', 'V')], [('есть', 'быть', 'V')], [('прижился', 'приживаться', 'V')]]


In [11]:
%%notify
verbsDict = dict()
for sentence in mystemVerbs:
    for wordTuple in sentence:
        verbsDict[wordTuple[0]] = wordTuple[1]

<IPython.core.display.Javascript object>

In [12]:
verbsDict['чувствую']

'чувствовать'

In [13]:
len(verbsDict.keys())

7657

In [15]:
inputFile = 'Data//OpenCorpora//opencorpora lemmas cleaned//text.txt'
file_obj = open(inputFile, 'r', encoding="utf8")
text = file_obj.read()
file_obj.close()

%notify -m "END READING"

for key in verbsDict.keys():
    text = text.replace(' ' + key + ' ', ' ' + verbsDict[key] + ' ')
    
%notify -m "END REPLACING"

outputFile = 'Data//OpenCorpora//opencorpora lemmas cleaned//text_modified.txt'
file_obj = open(outputFile, 'w', encoding="utf8")
file_obj.write(text)
file_obj.close()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Обработка ГИКРЯ

In [1]:
filesList = ['0751_joined_jj.out.gykMSD.txt', 
             '0752_joined_jj.out.gykMSD.txt', 
             '0752_joined_jj.out.gykMSD6466.txt',
             '0753_joined_jj.out.gykMSD.txt',
             '0753_joined_jj.out.gykMSD6936.txt',
             '0760_joined_vk.out.gykMSD.txt',
             '0760_joined_vk.out.gykMSD6270.txt',
             '0761_joined_vk.out.gykMSD.txt',
             '0761_joined_vk.out.gykMSD5875.txt',
             '0762_joined_vk.out.gykMSD.txt',
             '0762_joined_vk.out.gykMSD6173.txt'
            ]

In [1]:
def read_from_file(path, filename):
    inputFile = path + filename
    file_obj = open(inputFile, 'r', encoding="utf8")
    sentencesList = list()
    singleSentence = ' '

    index = 0
    for line in file_obj:
        if index < 2:
            index += 1
            continue
    
        if line == '\n':
            singleSentence = singleSentence + '\n'
            sentencesList.append(singleSentence)
            singleSentence = ' '
            continue

        wordInfo = line.split('\t')

        if len(wordInfo) < 6:
            continue

        lemma = wordInfo[3].rstrip()
        mark = wordInfo[4][0]
        if lemma == '[#UnknownWord]' or mark not in ['N', 'V', 'A', 'R']:
            continue
        else:
            lemma = lemma[:-1]
            lemma = lemma[1:]

        singleSentence = singleSentence + lemma + ' '
    
    file_obj.close()
    return sentencesList

In [14]:
len(sentencesListAll)

110288

In [4]:
for filename in filesList:
    sentencesList = read_from_file('Data//ГИКРЯ//GOLD_1.2_release//' , filename)
    # удаление предложений с малым количеством слов (с тремя)
    sentencesList = [x for x in sentencesList if len(x.split(' ')) > 5]
    write_to_file('Data//ГИКРЯ//Lemmatised texts//', filename, sentencesList)

In [7]:
len(sentencesListAll)

109922

In [2]:
sentencesListAll = list()
for file in filesList:
    filename = 'Data//ГИКРЯ//Lemmatised texts//' + file
    file_obj = open(filename, 'r', encoding="utf8")
    for line in file_obj:
        linetext = line.replace('_', ' ').lower()
        if linetext not in sentencesListAll:
            sentencesListAll.append(linetext)
    file_obj.close()

In [7]:
write_to_file('Data//ГИКРЯ//Lemmatised texts//', 'all_texts.txt', sentencesListAll)

## Combine texts

In [10]:
goldText = 'Data//ГИКРЯ//Lemmatised texts//all_texts.txt'
opencorporaText = 'Data//OpenCorpora//opencorpora lemmas cleaned//text_modified_cleaned.txt'

sentencesList = list()

file_obj = open(opencorporaText, 'r', encoding="utf8")
for line in file_obj:
    sentencesList.append(line)
file_obj.close()

file_obj = open(goldText, 'r', encoding="utf8")
for line in file_obj:
    sentencesList.append(line)
file_obj.close()

sentencesList = [x for x in sentencesList if len(x.split(' ')) > 5]
print(len(sentencesList))
write_to_file('Data\\', 'corpus.txt', sentencesList)

197450
